# **Company Profile Website AI Generator**
## with LLM Model "IBM Granite Instruct" using Replicate Platform

### **A Capstone project for IBM SkillsBuild: Code Generation and Optimization Using IBM Granite, collaborating with Hacktiv8 Indonesia**

## Created by:
### Peter Kurniawan Oktavianus
### https://www.linkedin.com/in/peter-kurniawan/

##Install Library dan Replicate Authentication

In [16]:
!pip install -U replicate

In [17]:
import os
from getpass import getpass
import replicate
from google.colab import userdata

# load replicate API Token dari google colab
os.environ["REPLICATE_API_TOKEN"] = userdata.get('REPLICATE_API_TOKEN')

# jika mau input API Token sendiri, bisa uncomment code ini
# os.environ["REPLICATE_API_TOKEN"] = input("Enter your Replicate API token: ")

## Inisialisasi model untuk Replicate Model Wrapper

In [18]:
class ReplicateModelWrapper:
    def __init__(self, model_name, **default_input):
        self.model_name = model_name
        self.default_input = default_input

    def invoke(self, prompt: str) -> str:
        input_args = self.default_input.copy()
        input_args["prompt"] = prompt
        output = replicate.run(self.model_name, input=input_args)
        # Handle potential FileOutput objects in the output list
        processed_output = []
        for item in output:
            if hasattr(item, 'url'): # Check if it's a FileOutput object (or similar)
                processed_output.append(item.url) # Append the URL or relevant attribute
            else:
                processed_output.append(str(item)) # Convert other items to string
        return "".join(processed_output)


# Model Parameter Tuning

# Model Granite Code Instruct with 8 Billion Parameters
model = ReplicateModelWrapper(
    "ibm-granite/granite-8b-code-instruct-128k",
    max_tokens=2000,
    temperature=0.2,
    top_p=1.0,
    top_k=1,
    presence_penalty=0,
    frequency_penalty=0,
    stop_sequences="</html>", #Karena output berupa HTML Code, maka diakhir tag </html>
)

###One Shot Prompt Example

In [19]:
example = {
    "question": "Create a professional company profile landing page with modern design",
    "context": "Build a complete HTML page for a technology company with header, hero section, services, stats, and footer. Return only the complete HTML file including all CSS and JavaScript, starting with <!DOCTYPE html> and ending with </html>.",
    "output": """
      <!DOCTYPE html>
        <html lang="en">
          <head>
          <meta charset="UTF-8"/>
          <title>TechCorp</title>
          <style>*{margin:0;padding:0}html,body{width:100%;height:100%;font-family:sans-serif;text-align:center;margin:0;padding:0}body{display:flex;flex-direction:column;min-height:100vh}header{background:#667eea;color:#fff;padding:20px;display:flex;justify-content:space-between;align-items:center;font-size:24px}a{color:#fff;text-decoration:none;margin:0 10px;font-size:20px}.hero{background:#764ba2;color:#fff;padding:120px 20px;height:80px;display:flex;flex-direction:column;justify-content:center}.hero h1{font-size:48px;margin-bottom:15px}.hero p{font-size:22px;margin-bottom:20px}button{background:#ff6b6b;color:#fff;border:none;padding:15px 30px;margin-top:15px;border-radius:6px;font-size:18px;cursor:pointer}.svc{background:#f8f9fa;padding:80px 20px;height:100px;display:flex;justify-content:center;align-items:center;flex-direction:column;flex:1}.svc h2{font-size:36px;margin-bottom:40px}.card{display:inline-block;background:#fff;padding:30px;margin:15px;border-radius:8px;box-shadow:0 2px 6px #ccc;width:140px;font-size:18px}.stats{background:#333;color:#fff;padding:40px;height:20vh;display:flex;justify-content:center;align-items:center}.st{display:inline-block;color:#ff6b6b;margin:0 25px;font-size:24px;text-align:center}.st .number{font-size:32px;font-weight:bold;display:block;margin-bottom:5px}footer{background:#222;color:#fff;padding:20px;font-size:16px;display:flex;justify-content:center;align-items:center}</style>
          </head>
          <body>
          <header>
            <b>TechCorp</b>
            <nav>
              <a href="#">Home</a>
              <a href="#">Services</a>
              <a href="#">About</a>
              <a href="#">Contact</a>
            </nav>
          </header>
          <div class="hero">
            <h1>Transform Digitally</h1>
            <p>Innovative solutions.</p>
            <button>Explore</button>
          </div>
          <div class="svc">
            <h2>Services</h2>
            <div>
              <div class="card">💻<br/>Web</div>
              <div class="card">📱<br/>Apps</div>
              <div class="card">☁️<br/>Cloud</div>
            </div>
          </div>
          <div class="stats">
            <div class="st"><span class="number">150</span>Projects</div>
            <div class="st"><span class="number">50</span>Clients</div>
            <div class="st"><span class="number">5</span>Years</div>
          </div>
          <footer>©2024 TechCorp</footer>
          </body>
        </html>
"""
}

## Function untuk format One Shot Prompt
### dengan parameter context, question, company info, dan example dari input user

In [20]:
def oneshot_prompt(context, question, company_info, example):
    """
    Creates a one-shot prompt for the model, using a single example to guide output generation.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    - company_info: dict, company information to include in the output
    - example: dict, single example containing 'question', 'context', and 'output' as keys
    Returns:
    - str, the formatted one-shot prompt
    """

    # Format company information
    company_details = f"Company: {company_info.get('name')}, Industry: {company_info.get('industry')}"

    # Format the example for the prompt
    formatted_example = f"""
    Example:
    User Question: {example['question']}
    Context: {example['context']}
    Model Output: {example['output']}
    """

    # Construct the one-shot prompt
    prompt = f"""
    You are an experienced web developer with 15 years of experience creating modern, responsive websites.
    Your task is to generate complete HTML pages with embedded CSS and JavaScript
    based on the provided context and user question.

    Here is an example of a similar task you have completed successfully:
    {formatted_example}

    Now, using this example as a reference, generate code for the following task:

    Context: {context}
    User Question: {question}
    Company Information: {company_details}

    INSTRUCTIONS:
    - Output ONLY the complete HTML code for the page.
    - Do NOT include any explanation, comments, or text before or after the code.
    - You may change colors, but use a palette that matches the style/theme of the provided example.
    - Do NOT remove any main sections shown in the example.
    - The output must start with <!DOCTYPE html> and end with </html>.
    - All CSS and JavaScript must be embedded in the HTML.
    """
    return prompt


## Function untuk generate AI model response
### berdasarkan One Shot Prompt yang sudah diformat

In [21]:
def get_answer_using_oneshot(context, question, company_info, example):
    """
    Generates the response from the model based on a one-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - company_info: dict, company information to include in the output
    - example: dict, single example to guide the model

    Returns:
    - str, the generated result complete HTML file including all CSS and JavaScript, starting with <!DOCTYPE html> and ending with </html>.
    """
    prompt = oneshot_prompt(context, question, company_info, example)
    result = model.invoke(prompt)

    return result

## Contoh penggunaan AI Model IBM Granite dengan One-Shot Prompting
### **Bagian ini customizable. Bisa diganti-ganti website company profile apa yang mau dibuat sesuai keinginan user.**

In [58]:
# Example usage
context = "Create a professional company profile landing page with modern design"
question = "Build a complete HTML page for a coding bootcamp company with header (Indonesia's Finest Coding Bootcamp), hero section, services (Fullstack Engineer, iOS/Android Developer, DevOps Engineer), stats (Students, Cities, Years), and footer sections. The stats section must be present, just like in the example. You may change colors, but use a palette that matches the style/theme of the company info. Do NOT remove any main sections shown in the example."
company_info = {"name": "Hacktiv8", "industry": "Coding Bootcamp"}
# Generate and display the HTML code for the landing page
code_generation_result = get_answer_using_oneshot(context, question, company_info, example)
print(f"Generated HTML Code:\n{code_generation_result}")

Generated HTML Code:
<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="UTF-8"/>
  <title>Hacktiv8</title>
  <style>*{margin:0;padding:0}html,body{width:100%;height:100%;font-family:sans-serif;text-align:center;margin:0;padding:0}body{display:flex;flex-direction:column;min-height:100vh}header{background:#008080;color:#fff;padding:20px;display:flex;justify-content:space-between;align-items:center;font-size:24px}a{color:#fff;text-decoration:none;margin:0 10px;font-size:20px}.hero{background:#008080;color:#fff;padding:120px 20px;height:80px;display:flex;flex-direction:column;justify-content:center}.hero h1{font-size:48px;margin-bottom:15px}.hero p{font-size:22px;margin-bottom:20px}button{background:#ff6b6b;color:#fff;border:none;padding:15px 30px;margin-top:15px;border-radius:6px;font-size:18px;cursor:pointer}.svc{background:#f8f9fa;padding:80px 20px;height:100px;display:flex;justify-content:center;align-items:center;flex-direction:column;flex:1}.svc h2{font-size:36px;margin-bottom

#### Count Tokens Code HTML yang sudah digenerate

In [59]:
!pip install tiktoken

In [60]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")
print("Token count:", len(encoding.encode(code_generation_result)))

Token count: 735


## Tampilkan Hasil Website yang sudah digenerate IBM Granite

In [61]:
from IPython.display import HTML, display

display(HTML(code_generation_result))

###One Shot Prompt Refined Example
#### Merupakan Example HTML yang sudah ditingkatkan fiturnya agar stats pada bagian bawah website bisa ada animasi perubahan angka.

In [62]:
refinement_example = {
    "question": "Enhance the stats section to have animated counters that increase when the section comes into view",
    "context": "Add intersection observer and counter animation to make numbers count up from 0 to their target values with smooth animation",
    "output": """
    // update div tags with data-target:
            <div class="stats">
              <div class="st"><span class="number" data-target="150">0</span>Projects</div>
              <div class="st"><span class="number" data-target="50">0</span>Clients</div>
              <div class="st"><span class="number" data-target="5">0</span>Years</div>
            </div>

    //update script to add function animateCounter(element):
        <script>
            function animateCounter(element) {
                const target = parseInt(element.getAttribute('data-target'));
                const increment = target / 50;
                let current = 0;

                const updateCounter = () => {
                    if (current < target) {
                        current += increment;
                        element.textContent = Math.ceil(current);
                        requestAnimationFrame(updateCounter);
                    } else {
                        element.textContent = target;
                    }
                };

                updateCounter();
            }

            const statNumbers = document.querySelectorAll('.number');
            const countObserver = new IntersectionObserver((entries) => {
                entries.forEach((entry) => {
                    if (entry.isIntersecting) {
                        animateCounter(entry.target);
                        countObserver.unobserve(entry.target);
                    }
                });
            }, {threshold: 0.5});

            statNumbers.forEach((stat) => {
                countObserver.observe(stat);
            });
        </script>
    """
}

## Function untuk format Refined One Shot Prompt
### dengan parameter previous HTML otuput, context, question, dan example (dari contoh refined prompt sebelumnya).

In [63]:
def refine_output_with_oneshot(previous_output, context, question, example):
    """
    Refines a previously generated output by applying additional user-requested changes
    using a one-shot prompt technique.

    Parameters:
    - previous_output: str, the HTML code generated from the initial prompt.
    - context: str, the context for the refinement task.
    - question: str, the user-requested changes to be applied.
    - example: dict, single example of similar refinement for one-shot learning.

    Returns:
    - str, the refined HTML code generated by the model.
    """

    # Format the example for the prompt
    formatted_refinement_example = f"""
    Example:
    User Question: {refinement_example['question']}
    Context: {refinement_example['context']}
    Model Output: {refinement_example['output']}
    """

    prompt = f"""
    You are an experienced web developer with 15 years of experience creating modern, responsive websites.
    Your task is to enhance existing HTML code by applying additional styling and functionality changes
    based on the provided context and user question.

    Here is an example of a similar refinement task you have completed successfully:
    {formatted_refinement_example}

    Now, I have an existing HTML page that needs enhancement. Please refine the following HTML code based on the additional requirements:

    Current HTML Code:
    {code_generation_result}

    Context: {context}
    User Question: {question}

    INSTRUCTIONS:
    Output ONLY the complete HTML code for the page.
    - Do NOT include any explanation, comments, or text before or after the code.
    - You may change colors, but use a palette that matches the style/theme of the provided example.
    - Do NOT remove any main sections shown in the example.
    - The output must start with <!DOCTYPE html> and end with </html>.
    - All CSS and JavaScript must be embedded in the HTML.
    - All existing functionality is preserved
    - New features are seamlessly integrated
    """

    # Use the model to generate the refined code
    result = model.invoke(prompt)
    return result

## Contoh penggunaan Refined One-Shot Prompting dengan IBM Granite
### ***Bagian ini customizable. Bisa diganti-ganti website company profile apa yang mau dibuat sesuai keinginan user.***

In [64]:
# Example usage
context = "Enhanced User Interaction with Animated Statistics"
question = "Enhance the stats section to have animated counters that increase when the section comes into view, similar to the example provided. Return only the complete HTML file including all CSS and JavaScript, starting with <!DOCTYPE html> and ending with </html>."
# Generate and display the refined HTML code
code_generation_refined_result = refine_output_with_oneshot(code_generation_result, context, question, refinement_example)
print(f"Refined HTML Code:\n{code_generation_refined_result}")

Refined HTML Code:
<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="UTF-8"/>
  <title>Hacktiv8</title>
  <style>*{margin:0;padding:0}html,body{width:100%;height:100%;font-family:sans-serif;text-align:center;margin:0;padding:0}body{display:flex;flex-direction:column;min-height:100vh}header{background:#008080;color:#fff;padding:20px;display:flex;justify-content:space-between;align-items:center;font-size:24px}a{color:#fff;text-decoration:none;margin:0 10px;font-size:20px}.hero{background:#008080;color:#fff;padding:120px 20px;height:80px;display:flex;flex-direction:column;justify-content:center}.hero h1{font-size:48px;margin-bottom:15px}.hero p{font-size:22px;margin-bottom:20px}button{background:#ff6b6b;color:#fff;border:none;padding:15px 30px;margin-top:15px;border-radius:6px;font-size:18px;cursor:pointer}.svc{background:#f8f9fa;padding:80px 20px;height:100px;display:flex;justify-content:center;align-items:center;flex-direction:column;flex:1}.svc h2{font-size:36px;margin-bottom:4

#### Count Tokens Code HTML yang sudah digenerate

In [65]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")
print("Token count:", len(encoding.encode(code_generation_result)))

Token count: 735


## Tampilkan Hasil Refined Website yang digenerate IBM Granite

In [66]:
from IPython.display import HTML, display

display(HTML(code_generation_refined_result))